In [4]:
from pylabel import importer
import os
import shutil
import random
import uuid
from PIL import Image

### Convert XML file to TXT Yolo format

In [2]:
dataset = importer.ImportVOC(path='Legacy/BAPDetectionV12/annotations')
dataset.export.ExportToYoloV5()

Exporting YOLO files...: 100%|██████████| 1079/1079 [00:01<00:00, 1041.75it/s]


['training\\dataset.yaml',
 'training\\labels\\04Mar1-(100)detect1detect1.txt',
 'training\\labels\\04Mar1-(101)detect1detect1.txt',
 'training\\labels\\04Mar1-(102)detect1detect1.txt',
 'training\\labels\\04Mar1-(103)detect1detect1.txt',
 'training\\labels\\04Mar1-(105)detect1detect1.txt',
 'training\\labels\\04Mar1-(107)detect1detect1.txt',
 'training\\labels\\04Mar1-(109)detect1detect1.txt',
 'training\\labels\\04Mar1-(11)detect1detect1.txt',
 'training\\labels\\04Mar1-(111)detect1detect1.txt',
 'training\\labels\\04Mar1-(112)detect1detect1.txt',
 'training\\labels\\04Mar1-(113)detect1detect1.txt',
 'training\\labels\\04Mar1-(118)detect1detect1.txt',
 'training\\labels\\04Mar1-(119)detect1detect1.txt',
 'training\\labels\\04Mar1-(121)detect1detect1.txt',
 'training\\labels\\04Mar1-(130)detect1detect1.txt',
 'training\\labels\\04Mar1-(131)detect1detect1.txt',
 'training\\labels\\04Mar1-(132)detect1detect1.txt',
 'training\\labels\\04Mar1-(133)detect1detect1.txt',
 'training\\labels\\

### Split into train and val

In [7]:
#os.mkdir('validation')
os.mkdir(os.path.join('bap','validation','images'))
os.mkdir(os.path.join('bap','validation','labels'))

In [2]:
filenames = os.listdir(os.path.join('bap','training','images'))
random.shuffle(filenames)
for filename in filenames:
    if '.txt' in filename: 
      shutil.move(os.path.join('bap','training','images',filename), os.path.join('bap','training','labels',filename))

In [7]:
train_filenames = os.listdir(os.path.join('Datasets','Expandidas', 'training', 'labels'))
for filename in train_filenames:
    with open(os.path.join('Datasets','Expandidas', 'training', 'labels', filename), 'r') as file:
        lines = file.readlines()
    lines = [line.replace('3', '0', 2) if line.startswith('3') else line for line in lines]
    with open(os.path.join('Datasets','Expandidas', 'training', 'labels', filename), 'w') as file:
        file.writelines(lines)


In [5]:
valid_filenames = os.listdir(os.path.join('Datasets','Expandidas', 'validation', 'labels'))
for filename in valid_filenames:
    with open(os.path.join('Datasets','Expandidas', 'validation', 'labels', filename), 'r') as file:
        lines = file.readlines()
    lines = [line.replace('3', '0', 2) if line.startswith('3') else line for line in lines]
    with open(os.path.join('Datasets','Expandidas', 'validation', 'labels', filename), 'w') as file:
        file.writelines(lines)

In [10]:
label_filenames = os.listdir(os.path.join('bap','training','labels'))
img_filenames = os.listdir(os.path.join('bap','training','images'))
for img_filename in img_filenames:
    lbl_filename = img_filename[:-4] + '.txt'
    if lbl_filename not in label_filenames:
        f = open(os.path.join('bap','training','labels',lbl_filename), "x")

In [11]:
train_split = 0.2
filenames = os.listdir(os.path.join('bap','training','images'))

random.shuffle(filenames)
for filename in filenames[:int(len(filenames)*train_split)]:
    label_filename = filename[:-4] + '.txt'
    shutil.move(os.path.join('bap','training','images',filename), os.path.join('bap','validation','images',filename))
    shutil.move(os.path.join('bap','training','labels',label_filename), os.path.join('bap','validation','labels',label_filename))

In [4]:
image_folder = os.path.join('post', 'training', 'images')
label_folder = os.path.join('post', 'training', 'labels')
output_folder = os.path.join('bap', 'training', 'images')

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for filename in os.listdir(image_folder):
    if filename.endswith('.jpg'):
        # open image file
        img_path = os.path.join(image_folder, filename)
        img = Image.open(img_path)
        
        # get size of the image
        img_width, img_height = img.size
        
        # open corresponding label file
        label_file = filename[:-4] + '.txt'
        label_path = os.path.join(label_folder, label_file)
        
        with open(label_path, 'r') as f:
            # each line represents one bounding box
            for line in f:
                data = line.strip().split()

                # convert YOLO center and width/height to pixel coordinates
                x_center = float(data[1]) * img_width
                y_center = float(data[2]) * img_height
                width = float(data[3]) * img_width
                height = float(data[4]) * img_height

                # calculate the top left and bottom right points of the bounding box
                top_left_x = x_center - (width / 2)
                top_left_y = y_center - (height / 2)
                bottom_right_x = x_center + (width / 2)
                bottom_right_y = y_center + (height / 2)

                # crop the image and save it
                cropped_img = img.crop((top_left_x, top_left_y, bottom_right_x, bottom_right_y))
                output_file = os.path.join(output_folder, str(uuid.uuid4()) + '.jpg')
                cropped_img.save(output_file)